In [1]:
from tensorflow.python.client import device_lib
#print(device_lib.list_local_devices())

import tensorflow as tf
print("Num GPUs Available: ", len(tf.config.experimental.list_physical_devices('GPU')))
physical_devices = tf.config.list_physical_devices('GPU')
#tf.config.experimental.set_memory_growth(physical_devices[0], enable=True)

Num GPUs Available:  1


In [95]:
from tensorflow import keras
from tensorflow.keras import layers, models, metrics
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras import optimizers
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.applications import *
from tensorflow.keras.layers import Rescaling,BatchNormalization, Conv2D, MaxPooling2D, Flatten, Dense, Input, Dropout, GlobalAveragePooling2D 
from sklearn.model_selection import train_test_split

In [3]:
import cv2
import numpy as np
import tensorflow as tf
from tensorflow import keras


In [4]:
data="../../data/data_individual/train/"

In [5]:
X=[]
Y=[]

In [126]:
complete_data_train=tf.keras.preprocessing.image_dataset_from_directory(data,image_size=(256,512),shuffle=True,seed=69,subset="training",labels="inferred",
    label_mode="categorical",validation_split=0.2,batch_size=8)

Found 1649 files belonging to 128 classes.
Using 1320 files for training.


In [127]:
complete_data_val=tf.keras.preprocessing.image_dataset_from_directory(data,image_size=(256,512),shuffle=True,seed=69,subset="validation",labels="inferred",
    label_mode="categorical",validation_split=0.2,batch_size=8)

Found 1649 files belonging to 128 classes.
Using 329 files for validation.


In [128]:
print(complete_data_val)

<BatchDataset shapes: ((None, 256, 512, 3), (None, 128)), types: (tf.float32, tf.float32)>


In [129]:
input_shape=(256,512,3)

In [139]:
base_model = keras.applications.Xception(
    weights="imagenet",  # Load weights pre-trained on ImageNet.
    input_shape=(256, 512, 3),
    include_top=False,
)

In [140]:
base_model.trainable = False

In [141]:
inputs = keras.Input(shape=(256, 512, 3))
x=inputs
x = layers.Rescaling(1./255)(x)

In [142]:
x = base_model(x, training=False)
x = keras.layers.GlobalAveragePooling2D()(x)
x = keras.layers.Dropout(0.2)(x)  # Regularize with dropout
outputs = keras.layers.Dense(128)(x)
model = keras.Model(inputs, outputs)

In [143]:
model.summary()

Model: "model_4"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_27 (InputLayer)       [(None, 256, 512, 3)]     0         
                                                                 
 rescaling_7 (Rescaling)     (None, 256, 512, 3)       0         
                                                                 
 xception (Functional)       (None, 8, 16, 2048)       20861480  
                                                                 
 global_average_pooling2d_7   (None, 2048)             0         
 (GlobalAveragePooling2D)                                        
                                                                 
 dropout_7 (Dropout)         (None, 2048)              0         
                                                                 
 dense_7 (Dense)             (None, 128)               262272    
                                                           

In [145]:
model.compile(
    optimizer=keras.optimizers.Adam(),
#     loss=keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    loss="categorical_crossentropy",
    metrics=["Accuracy"],
)

epochs = 20

In [146]:
base_model_=model.fit(complete_data_train, batch_size=8,epochs=epochs, validation_data=complete_data_val)

Epoch 1/20
165/165 [==============================] - 19s 106ms/step - loss: 9.2905 - Accuracy: 0.0258 - val_loss: 10.2176 - val_Accuracy: 0.0456
Epoch 2/20
165/165 [==============================] - 17s 102ms/step - loss: 10.7328 - Accuracy: 0.0492 - val_loss: 11.0073 - val_Accuracy: 0.0334
Epoch 3/20
165/165 [==============================] - 17s 103ms/step - loss: 11.1444 - Accuracy: 0.0621 - val_loss: 11.7438 - val_Accuracy: 0.0426
Epoch 4/20
165/165 [==============================] - 17s 104ms/step - loss: 11.7540 - Accuracy: 0.0742 - val_loss: 12.0323 - val_Accuracy: 0.0213
Epoch 5/20
165/165 [==============================] - 17s 106ms/step - loss: 11.7896 - Accuracy: 0.0712 - val_loss: 11.6813 - val_Accuracy: 0.0182
Epoch 6/20
165/165 [==============================] - 17s 104ms/step - loss: 11.8475 - Accuracy: 0.0614 - val_loss: 10.8483 - val_Accuracy: 0.0182
Epoch 7/20
165/165 [==============================] - 17s 103ms/step - loss: 12.1547 - Accuracy: 0.0538 - val_loss: 11.

In [147]:
pwd

'D:\\projects\\Personal\\Fraudulent-Signature-Detection\\model\\multi_class'

In [148]:
model.save("../saved_models/multi_class_x_net_base_model.h5")

D:\projects\Jupyter\gpu-env\lib\site-packages\keras\engine\functional.py:1410: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  layer_config = serialize_layer_fn(layer)
